In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/final_project'

In [ ]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

from sklearn.metrics import accuracy_score, confusion_matrix

from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import pyplot
from mlxtend.plotting import plot_confusion_matrix


In [ ]:
# List of emotions on the dataset we want to include (using comments -#- to exlcude).

EMOTIONS=[
    'happy',
    'surprise',
    'neutral',
    'disgust',
    'sad',
    #'angry',
    #'fear'
]

In [ ]:
# Load dataset images into the DataFrame.

dataset = []

for emotion in EMOTIONS:
    print(emotion)
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}')
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalization of the data

X = np.array([MinMaxScaler().fit_transform(e) for e in df['pixels']])
lb = LabelBinarizer()
Y = lb.fit_transform(df['emotion'])

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_test.shape

In [ ]:
fig = pyplot.figure(1, (14,14))

k = 0
for label in sorted(df.emotion.unique()):
    for j in range(7):
        px = df[df.emotion==label].pixels.iloc[k]
        px = px.reshape(48, 48)

        k += 1
        ax = pyplot.subplot(7, 7, k)
        ax.imshow(px, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label)
        pyplot.tight_layout()

In [ ]:
# Load saved model

MODEL='models/modelConv.h5'

model = tf.keras.models.load_model(MODEL)

In [ ]:
# Confusional Matrix

Y_pred = model.predict(X_test)

Y_labels = lb.inverse_transform(Y_test)
Y_pred_labels = lb.inverse_transform(Y_pred)

m = confusion_matrix(Y_labels, Y_pred_labels)
plot_confusion_matrix(m, show_absolute=False, show_normed=True)
plt.show()

In [ ]:
for index, emotion in enumerate(EMOTIONS):
  print(index, '->', emotion)

In [ ]:
# Accuracy in every emotion with CNN model

class_accuracies = m.diagonal() / m.sum(axis=1)

for i, class_accuracy in enumerate(class_accuracies):
  print("Acc of", i, ":", class_accuracy)




**Vemos que los modelo que mejor Accuracy han tenido son**: 

* Happy 0.91
* Surprise 0.86,
* Neutral 0.83, 
* Disgust 0.68, 
* Sad 0.68 y 
* Angry 0.67.



**Entonces Eliminamos la emoción Fear y volvemos a entrenar el modelo**: 





In [ ]:

# Load saved model

MODEL='models/modelConv_cleaned.h5'

model = tf.keras.models.load_model(MODEL)



In [ ]:
# Confusional Matrix

Y_pred = model.predict(X_test)

Y_labels = lb.inverse_transform(Y_test)
Y_pred_labels = lb.inverse_transform(Y_pred)

m = confusion_matrix(Y_labels, Y_pred_labels)

plot_confusion_matrix(m, show_absolute=False, show_normed=True)
plt.show()

In [ ]:
# Accuracy in every emotion with CNN model

class_accuracies = m.diagonal() / m.sum(axis=1)

for i, class_accuracy in enumerate(class_accuracies):
  print("Acc of", i, ":", class_accuracy)

In [ ]:
# Prediction Vs ground truth

pred_fail_idx = np.where(Y_labels != Y_pred_labels)

In [ ]:
# Showing prediction Vs ground truth

SHOW=[8,8]

fig, ax = plt.subplots(SHOW[0], SHOW[1], sharex=True, sharey=True, figsize=(25,25))

for i in range(SHOW[0]):
    for j in range(SHOW[1]):
        idx = pred_fail_idx[0][i*SHOW[0]+j]
        ax[i][j].imshow(X[idx], cmap='gray')
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])   
        ax[i][j].set_title(f'Gt: {Y_labels[idx]} || Pred: {Y_pred_labels[idx]}')
    
plt.show()

In [ ]:
def miss_classified(emotion):
    missed_index = np.where((Y_test != Y_pred) & (Y_test == (df['emotion'].unique())))[0]
    print(f"total {len(missed_index)} miss labels out of {len(np.where(Y_test == (df['emotion'].unique()))[0])} for emotion {emotion}")


In [ ]:
miss_classified("Happy")